Import the necessary packages for data analysis.

In [8]:
import numpy as np
import pandas as pd
import matplotlib as plt

Load the data and have a first glance at it.

In [7]:
try:
    csv_path = "../COVID19-modelling/data/covid_pol_voivodeship.csv"
    load_csv = pd.read_csv(csv_path)
    print("The data was loaded correctly")
except NameError:
    print("The path", csv_path, "cannot be found")

load_csv

The data was loaded correctly


,Date (CET),Lower Silesia (DS),Kuyavia-Pomerania (KP),Lubusz (LB),Łódź (LD),Lublin (LU),Lesser Poland (MA),Masovia (MZ; Warsaw),Opole (OP),Podlaskie (PD),Subcarpathian (PK),Pomerania (PM),Holy Cross (SK),Silesia (SL),Warmia–Masuria (WN),Greater Poland (WP),West Pomerania (ZP),Poland daily,Poland total,Sources
0,4 March 2020,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,[714]
1,5 March 2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,NaN
2,6 March 2020,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,2,4,5,[715]
3,7 March 2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,1,6,[17]
4,8 March 2020,1,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,5,11,[19][22]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,27 August 2020,28,24,12,47,14,233,107,16,12,33,71,18,118,51,92,11,887,64 689,[829]
177,28 August 2020,30,38,9,39,27,139,130,15,5,70,42,19,104,22,92,10,791,65 480,[830]
178,Infections per voivodeship,3995,1268,660,5196,1484,7128,9108,1603,1262,2331,2371,1380,20236,857,5505,1096,NaN,65 480,NaN
179,Date (CET),Lower Silesia (DS),Kuyavia-Pomerania (KP),Lubusz (LB),Łódź (LD),Lublin (LU),Lesser Poland (MA),Masovia (MZ; Warsaw),Opole (OP),Podlaskie (PD),Subcarpathian (PK),Pomerania (PM),Holy Cross (SK),Silesia (SL),Warmia–Masuria (WN),Greater Poland (WP),West Pomerania (ZP),Poland daily,Poland total,Sources


Reorganize and clean the data. Prepare it for an analysis.

In [26]:
lesser_pol_data = [load_csv["Date (CET)"], load_csv["Lesser Poland (MA)"]]
headers = ["Date", "Number of infections in Lesser Poland"]

df3 = pd.concat(lesser_pol_data, axis=1, keys=headers)

df3

df4 = df3.drop([180, 179, 178])
df4

,Date,Number of infections in Lesser Poland
0,4 March 2020,NaN
1,5 March 2020,NaN
2,6 March 2020,NaN
3,7 March 2020,NaN
4,8 March 2020,NaN
...,...,...
173,24 August 2020,61
174,25 August 2020,141
175,26 August 2020,118
176,27 August 2020,233
